In [4]:
import os
import nbimporter
import numpy as np
from random import randint

In [129]:
def stroke_removal(drawing, percent=0.35, alpha=0.5, beta=0.5):
    num = int(len(drawing) * percent)
    temp = drawing.copy()
    pi = []
    for i, stroke in enumerate(drawing):
        pi.append(np.exp(alpha * (i+1)) / np.exp(beta * len(stroke[0])))
    pi = pi / np.array(pi).sum()
    order_to_remove = np.argsort(pi)[::-1]
    for i in range(0, num):
        del temp[order_to_remove[i]]
    return temp